In [4]:
from knowknow import *

In [12]:
from load_db import db

In [13]:
#db = Dataset('sociology-wos-73b')
#top_1970 = stats.top_decade_stratified(db, 'c', 0.01, yRange=(1970,1980), debug=True)

In [14]:
total = 0
died = 0
died1 = 0

#top_1970_n = top_1970['name']
min_top_cited = float('inf')
all_avgbet = []
all_totals = []

for name in db.items('c'):
    tt = db.trend('c', name)
    
    all_totals.append( (name, tt.sum_between(1970,1980)) )
    
    # only count among works that are cited at least 5 times in 1970-1980
    range1 = tt.sum_between(1970,1980) 
    if range1 < 5:
        continue        
        
    total += 1
    
    #range1 = min(min_top_cited, )
    B,D = tt.births_deaths(death_cutoff=0.1)
    B1,D1 = tt.births_deaths(death_cutoff=0)
    
    # did they die?
    if len(D):
        if D[-1] >= 1970:
            died += 1
    # did they die?
    if len(D1):
        if D1[-1] >= 1970:
            died1 += 1
            
    # how are they doing now?
    ab = tt.avg_between(2005,2015)
    all_avgbet.append(ab)

loading variable sociology-wos-74b/ind ___ c from disk


In [15]:
prop1 = died / total

In [16]:
prop2 = died1 / total

In [17]:
prop3 = sum( np.array(all_avgbet) < 1e-8 ) / len(all_avgbet)

In [113]:
prop1, prop2, prop3

(0.6828232537706839, 0.6211744032801289, 0.4255381461414556)

In [18]:
prop1, prop2, prop3

(0.6828232537706839, 0.6211744032801289, 0.4255381461414556)

In [44]:
display(Markdown(f"""
{prop2:0.0%} of works that got at least 5 citations between 1970 and 1980
    went at least a decade without being cited.
{prop3:0.0%} were not cited whatsoever between 2005 and 2015.
"""))


62% of works that got at least 5 citations between 1970 and 1980
    went at least a decade without being cited.
43% were not cited whatsoever between 2005 and 2015.


In [49]:
cy = db.by('c','fy').cits

In [50]:
cited = set()

for (c,fy), C in cy.items():
    if C <= 0:
        continue
    if 1970 <= fy <= 1980:
        cited.add(c)

In [51]:
total_cited = len(cited)

In [52]:
total_cited

18543

In [53]:
total = 0
died = 0
for name in cited:
    total += 1
    B,D = db.trend('c', name).births_deaths(death_cutoff=0.1)
    
    if len(D):
        died += 1

In [54]:
prop2 = died / total

In [55]:
prop1, prop2

(0.6828232537706839, 0.7830987434611444)

In [56]:
cited_after_1990 = set()

In [57]:
for (c,fy), C in cy.items():
    if C <= 0:
        continue
    if 1990 < fy and c in cited:
        cited_after_1990.add(c)

In [60]:
p1 = 1 - len(cited_after_1990) / len(cited)

In [61]:
died = 0
total = 0

for c in cited:
    total += 1
    ct = db.trend('c', c)
    B,D =  ct.births_deaths( death_cutoff=0 )
    if len(D):
        died += 1

In [62]:
p2 = died/total

In [69]:
died = 0
total = 0

for c in cited:
    total += 1
    ct = db.trend('c', c)
    
    if ct.avg_between(2005,2015) >= 1:
        died += 1

In [70]:
p3 = died/total

In [71]:
p3

0.06347408725664672

In [72]:
all_avgbet = []

died = 0
total = 0

for c in db.items('c'):
    total += 1
    ct = db.trend('c', c)
    ab = ct.avg_between(2005,2015)
    all_avgbet.append(ab)

In [73]:
p4 = sum( np.array(all_avgbet) > 1 ) / len(all_avgbet)

In [74]:
p4

0.1293906595706512

In [75]:
display(Markdown(f"""

The lifespan of a cited work is well described in aggregate by a lognormal distribution.
The work reaches its maximum after a few years, declines slowly for 5-10 years, eventually recieving its last citation.

A few statistics should be sufficient to illustrate that cited works are almost surely headed to the grave, never to be revived.
There were {total_cited:,} works cited for the first time in the years 1970 to 1980.
{p1:0.0%} of these would not receive another citation after 1980.
{p2:0.0%} of works cited in the 70s has gone at least a decade without being cited at all.
And only {p3:0.0%} received at least 1 citation per year on average between 2005 and 2015, 
For comparison, {1-p4:0.0%} of works first cited between 2005 and 2015 received a yearly average of at least 1 citation.
"""))



The lifespan of a cited work is well described in aggregate by a lognormal distribution.
The work reaches its maximum after a few years, declines slowly for 5-10 years, eventually recieving its last citation.

A few statistics should be sufficient to illustrate that cited works are almost surely headed to the grave, never to be revived.
There were 18,543 works cited for the first time in the years 1970 to 1980.
19% of these would not receive another citation after 1980.
75% of works cited in the 70s has gone at least a decade without being cited at all.
And only 6% received at least 1 citation per year on average between 2005 and 2015, 
For comparison, 87% of works first cited between 2005 and 2015 received a yearly average of at least 1 citation.


# top 1%

In [76]:
len(all_totals)

83615

In [77]:
totns = np.array([x[1] for x in all_totals])

top1q = np.quantile(totns, 0.99)
top1 = [x for x,c in all_totals if c >= top1q]

In [78]:
all_avgbet = []

died = 0
total = 0

for c in top1:
    total += 1
    ct = db.trend('c', c)
    ab = ct.avg_between(2005,2015)
    all_avgbet.append(ab)

In [79]:
len(top1)

874

In [80]:
len(all_avgbet)

874

In [81]:
p1 = sum( np.array(all_avgbet) > 1 ) / len(all_avgbet)

In [84]:
display(Markdown(f"""
{len(all_totals):,} works in this dataset were cited at some point between 1970 and 1980.
The top 1% consists of {len(all_avgbet)} cited works which have at least {top1q} citations in this decade.
Of these {p1:0.1%} got at least 1 citation per year in 2005 - 2015.
For comparison, for all works cited at least 5 times in the 1970s this statistic is just {p3:0.0%}.
"""))


83,615 works in this dataset were cited at some point between 1970 and 1980.
The top 1% consists of 874 cited works which have at least 21.0 citations in this decade.
Of these 44.5% got at least 1 citation per year in 2005 - 2015.
For comparison, for all works cited at least 5 times in the 1970s this statistic is just 6%.


In [83]:
874/83615

0.010452669975482868